In [1]:
# Import PadreApp from a package
from padre.app import pypadre
# All interactions with padre should be done via the app

[WARNING] [13:13:22:pypadre - http] Authentication token is NONE. You need to authentication for user mgrani with your current password (or set a new user)


In [2]:
# Configuration of padreapp via a configuration file including backend, http, server authentication etc
"""
 from padre.app.padre_app import pypadre
 assert pypadre.offline
 pypadre.authenticate("test", "mgrani") # authentication sets the pypadre app into online mode
 pypadre.config.save()
 assert pypadre.offline == False
 # delete all datasets
 for ds in pypadre.datasets.list():
     pypadre.datasets.delete(ds)
 assert len([ds for ds in pypadre.datasets.list()])==0 # after deletion there should be no datasets
 # fetch downloads. This should include the datasets that are currently supported.
 downloads = pypadre.datasets.search_downloads("Iris", type="multivariate") # search for external datasets
 # user can manipulate downloads here
 downloaded = pypadre.datasets.download(downloads)
 for download in downloaded: 
    pypadre.datasets.put(ds)
 available = pypadre.datasets.list("Iris")
 assert downloads.equals(available) # the check might be more complex here, but both sets should be equal
 dataset = pypadre.datasets.get(available[0])
"""

'\n from padre.app.padre_app import pypadre\n assert pypadre.offline\n pypadre.authenticate("test", "mgrani") # authentication sets the pypadre app into online mode\n pypadre.config.save()\n assert pypadre.offline == False\n # delete all datasets\n for ds in pypadre.datasets.list():\n     pypadre.datasets.delete(ds)\n assert len([ds for ds in pypadre.datasets.list()])==0 # after deletion there should be no datasets\n # fetch downloads. This should include the datasets that are currently supported.\n downloads = pypadre.datasets.search_downloads("Iris", type="multivariate") # search for external datasets\n # user can manipulate downloads here\n downloaded = pypadre.datasets.download(downloads)\n for download in downloaded: \n    pypadre.datasets.put(ds)\n available = pypadre.datasets.list("Iris")\n assert downloads.equals(available) # the check might be more complex here, but both sets should be equal\n dataset = pypadre.datasets.get(available[0])\n'

In [3]:
# List all available datasets(Toy Datasets)
print(pypadre.datasets.list())

[]


In [4]:
# List all currently available local experiments
print(pypadre.experiments.list_experiments())

['JSON FILE TEST EXPERIMENT', 'Executor Test 1(Boston_House_Prices)', 'Executor Test 3', 'Executor Test 2', 'Executor Test 1(Diabetes)', 'Executor Test 1(Iris)']


In [5]:
# Create a simple experiment via experiment creator utilizing a dataset displayed. Run it with multiple parameters
pypadre.experiment_creator.clear_experiments()
workflow = pypadre.experiment_creator.create_test_pipeline(['isomap embedding'])
pypadre.experiment_creator.create_experiment(name='iPython Experiment1 ', 
                                           description='Simple iPython notebook experiment',
                                           workflow=workflow, 
                                           dataset_list=['Digits'])
pypadre.experiment_creator.execute_experiments()

# Aimed at new users for running already created experiments or running from JSON files

{'condition': False, 'source': <padre.experimentcreator.ExperimentCreator object at 0x7f4a4636c1d0>, 'message': 'iPython Experiment1  created successfully!'}
{'Step_0': {'algorithm': {'attributes': {'path': '.steps[0][1]'},
                          'value': 'isomap embedding'},
            'doc': {'attributes': {'path': '.steps[0][1].__doc__'},
                    'value': 'Isomap Embedding\n'
                             '\n'
                             '    Non-linear dimensionality reduction through '
                             'Isometric Mapping\n'
                             '\n'
                             '    Read more in the :ref:`User Guide '
                             '<isomap>`.\n'
                             '\n'
                             '    Parameters\n'
                             '    ----------\n'
                             '    n_neighbors : integer\n'
                             '        number of neighbors to consider for each '
                   

In [6]:
# Using decorators to run experiments
from padre.ds_import import load_sklearn_toys
from padre import *
from padre.app import pypadre
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

@Workflow(exp_name="Decorated Experiment",
          description="Test experiment with decorators and SVC")
def create_test_pipeline():
    estimators = [('clf', SVC(probability=True))]
    return Pipeline(estimators)


@Dataset(exp_name="Decorated Experiment")
def get_dataset():
    ds = [i for i in load_sklearn_toys()][2]
    return ds



In [7]:
ex = pypadre.experiments.run(decorated=True)
ex = run("Decorated Experiment")  # run the experiment and report
# Aimed at engineers and scientists for quick prototyping and trying out features

creating the workflow
INFO: 2019-02-04 13:13:28.267 Experiment<id:Decorated Experiment>:	start: phase=experiment
INFO: 2019-02-04 13:13:28.268 	Run<id:47854c3a-2876-11e9-bfa3-1860248c6d22;name:Decorated Experiment>:	start: phase=run
INFO: 2019-02-04 13:13:28.269 		Split<id:0_47854c3b-2876-11e9-bfa3-1860248c6d22;name:Decorated Experiment>:	start: phase=split
INFO: 2019-02-04 13:13:28.719 		Split<id:0_47854c3b-2876-11e9-bfa3-1860248c6d22;name:Decorated Experiment>:	stop: phase=split
INFO: 2019-02-04 13:13:28.730 	Run<id:47854c3a-2876-11e9-bfa3-1860248c6d22;name:Decorated Experiment>:	stop: phase=run
INFO: 2019-02-04 13:13:28.741 Experiment<id:Decorated Experiment>:	stop: phase=experiment
creating the workflow
INFO: 2019-02-04 13:13:28.809 Experiment<id:Decorated Experiment>:	start: phase=experiment
INFO: 2019-02-04 13:13:28.809 	Run<id:47854c3c-2876-11e9-bfa3-1860248c6d22;name:Decorated Experiment>:	start: phase=run
INFO: 2019-02-04 13:13:28.810 		Split<id:0_47854c3d-2876-11e9-bfa3-18602

In [8]:
# Creating and running experiments via pure code (example.py)
def create_test_pipeline():
    from sklearn.pipeline import Pipeline
    from sklearn.svm import SVC
    from sklearn.decomposition import PCA
    # estimators = [('reduce_dim', PCA()), ('clf', SVC())]
    estimators = [('clf', SVC(probability=True))]
    return Pipeline(estimators)



In [9]:
from padre.ds_import import load_sklearn_toys
from padre.core import Experiment
from padre.base import PadreLogger
from padre.eventhandler import add_logger

logger = PadreLogger()
logger.backend = pypadre.repository
add_logger(logger=logger)
ds = [i for i in load_sklearn_toys()][2]
ex = Experiment(name="Test Experiment SVM",
                    description="Testing Support Vector Machines via SKLearn Pipeline",
                    dataset=ds,
                    workflow=create_test_pipeline(), keep_splits=True, strategy="random")
ex.execute()

INFO: 2019-02-04 13:13:30.208 Experiment<id:Test Experiment SVM>:	start: phase=experiment
INFO: 2019-02-04 13:13:30.208 Experiment<id:Test Experiment SVM>:	start: phase=experiment
INFO: 2019-02-04 13:13:30.209 	Run<id:48ad7646-2876-11e9-bfa3-1860248c6d22;name:Test Experiment SVM>:	start: phase=run
INFO: 2019-02-04 13:13:30.209 	Run<id:48ad7646-2876-11e9-bfa3-1860248c6d22;name:Test Experiment SVM>:	start: phase=run
INFO: 2019-02-04 13:13:30.210 		Split<id:0_48ad7647-2876-11e9-bfa3-1860248c6d22;name:Test Experiment SVM>:	start: phase=split
INFO: 2019-02-04 13:13:30.210 		Split<id:0_48ad7647-2876-11e9-bfa3-1860248c6d22;name:Test Experiment SVM>:	start: phase=split
INFO: 2019-02-04 13:13:30.654 		Split<id:0_48ad7647-2876-11e9-bfa3-1860248c6d22;name:Test Experiment SVM>:	stop: phase=split
INFO: 2019-02-04 13:13:30.664 		Split<id:0_48ad7647-2876-11e9-bfa3-1860248c6d22;name:Test Experiment SVM>:	stop: phase=split
INFO: 2019-02-04 13:13:30.675 	Run<id:48ad7646-2876-11e9-bfa3-1860248c6d22;name:

In [12]:
# The PyTorch wrapper supports the use of User Defined Callbacks 
from padre.core.wrappers.wrapper_pytorch import WrapperPytorch, CallBack
class TestCallbacks(CallBack):
    def on_compute_loss(self, loss):
        print('Function on compute loss. Loss = {loss}'.format(loss=loss))

    def on_epoch_end(self, obj):
        print('Epoch ended')

    def on_epoch_start(self, obj):
        print('Epoch started')

    def on_iteration_start(self, obj):
        print('Iteration started')

    def on_iteration_end(self, obj):
        print('Iteration ended')

In [14]:
# Running an experiment with a custom wrapper
from sklearn.pipeline import Pipeline

# Define the parameters
params = dict()

import json
with open('../PyPaDRe/tests/proof_of_concept/Pytorch/classification.json') as json_data:
    params = json.load(json_data)

obj = WrapperPytorch(params=params)
obj.set_callbacks([TestCallbacks()])
estimators = [('clf', obj)]
workflow = Pipeline(estimators)
ds = [i for i in load_sklearn_toys()][4]

ex = Experiment(name="Torch",
                description="Testing Torch via SKLearn Pipeline",
                dataset=ds,
                workflow=workflow)
ex.execute()

End of constructor
INFO: 2019-02-04 13:16:34.167 Experiment<id:Torch>:	start: phase=experiment
INFO: 2019-02-04 13:16:34.168 Experiment<id:Torch>:	start: phase=experiment
INFO: 2019-02-04 13:16:34.169 	Run<id:b653738a-2876-11e9-bfa3-1860248c6d22;name:Torch>:	start: phase=run
INFO: 2019-02-04 13:16:34.170 	Run<id:b653738a-2876-11e9-bfa3-1860248c6d22;name:Torch>:	start: phase=run
INFO: 2019-02-04 13:16:34.170 		Split<id:0_b653738b-2876-11e9-bfa3-1860248c6d22;name:Torch>:	start: phase=split
INFO: 2019-02-04 13:16:34.171 		Split<id:0_b653738b-2876-11e9-bfa3-1860248c6d22;name:Torch>:	start: phase=split
Epoch started
Iteration started
Function on compute loss. Loss = 0.2794652901284985
1 0.2794652901284985
Iteration ended
Iteration started
Function on compute loss. Loss = 0.2766289490882206
2 0.2766289490882206
Iteration ended
Iteration started
Function on compute loss. Loss = 0.1308885145766014
3 0.1308885145766014
Iteration ended
Iteration started
Function on compute loss. Loss = 0.1799864

Function on compute loss. Loss = 0.18118859800847625
97 0.18118859800847625
Iteration ended
Iteration started
Function on compute loss. Loss = 0.22260282197417514
98 0.22260282197417514
Iteration ended
Iteration started
Function on compute loss. Loss = 0.21848139926931198
99 0.21848139926931198
Iteration ended
Iteration started
Function on compute loss. Loss = 0.23587950284968978
100 0.23587950284968978
Iteration ended
INFO: 2019-02-04 13:16:34.298 		Split<id:0_b653738b-2876-11e9-bfa3-1860248c6d22;name:Torch>:	stop: phase=split
INFO: 2019-02-04 13:16:34.299 		Split<id:0_b653738b-2876-11e9-bfa3-1860248c6d22;name:Torch>:	stop: phase=split
INFO: 2019-02-04 13:16:34.299 	Run<id:b653738a-2876-11e9-bfa3-1860248c6d22;name:Torch>:	stop: phase=run
INFO: 2019-02-04 13:16:34.299 	Run<id:b653738a-2876-11e9-bfa3-1860248c6d22;name:Torch>:	stop: phase=run
INFO: 2019-02-04 13:16:34.299 Experiment<id:Torch>:	stop: phase=experiment
INFO: 2019-02-04 13:16:34.299 Experiment<id:Torch>:	stop: phase=experime

/home/christofer/PycharmProjects/test_folder/venv/lib/python3.6/site-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for softmin has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [15]:
# Compare the metrics between experiments and also between runs(Displaying pandas dataframe)
from padre.experimentexecutor import ExperimentExecutor
# Create an experiment with multiple runs
params_pca = {'num_components': [2, 3, 4, 5, 6]}
params_svr = {'C': [0.5, 1.0, 1.5],
              'poly_degree': [1, 2, 3]}
params_dict = {'SVR': params_svr, 'pca': params_pca}
workflow = pypadre.experiment_creator.create_test_pipeline(['pca', 'SVR'])
params_dict = pypadre.experiment_creator.convert_alternate_estimator_names(params_dict)
pypadre.experiment_creator.clear_experiments('IPython Experiment 3')
pypadre.experiment_creator.create_experiment(name='IPython Experiment 3',
                                             description='Experiment to test the comparison of metrics',
                                             dataset_list=['Boston_House_Prices'],
                                             workflow=workflow,
                                             params=params_dict
                                             )
experiments_executor = ExperimentExecutor(experiments=pypadre.experiment_creator.createExperimentList())
experiments_executor.execute(local_run=True, threads=1)
from padre.metrics import CompareMetrics
metrics = CompareMetrics(experiments_list=experiments_executor.experiments)
print(metrics.show_metrics())

{'condition': False, 'source': <padre.experimentcreator.ExperimentCreator object at 0x7f4a4636c1d0>, 'message': 'IPython Experiment 3 created successfully!'}
Sequential Execution
{'message': 'Executing experiment: iPython Experiment1 '}
{'Step_0': {'algorithm': {'attributes': {'path': '.steps[0][1]'},
                          'value': 'isomap embedding'},
            'doc': {'attributes': {'path': '.steps[0][1].__doc__'},
                    'value': 'Isomap Embedding\n'
                             '\n'
                             '    Non-linear dimensionality reduction through '
                             'Isometric Mapping\n'
                             '\n'
                             '    Read more in the :ref:`User Guide '
                             '<isomap>`.\n'
                             '\n'
                             '    Parameters\n'
                             '    ----------\n'
                             '    n_neighbors : integer\n'
                     

INFO: 2019-02-04 13:17:04.735 		Split<id:0_c81dde7b-2876-11e9-bfa3-1860248c6d22;name:iPython Experiment1 >:	stop: phase=split
INFO: 2019-02-04 13:17:04.735 		Split<id:0_c81dde7b-2876-11e9-bfa3-1860248c6d22;name:iPython Experiment1 >:	stop: phase=split
INFO: 2019-02-04 13:17:04.735 	Run<id:c81dde7a-2876-11e9-bfa3-1860248c6d22;name:iPython Experiment1 >:	stop: phase=run
INFO: 2019-02-04 13:17:04.735 	Run<id:c81dde7a-2876-11e9-bfa3-1860248c6d22;name:iPython Experiment1 >:	stop: phase=run
INFO: 2019-02-04 13:17:04.735 Experiment<id:iPython Experiment1 >:	stop: phase=experiment
INFO: 2019-02-04 13:17:04.735 Experiment<id:iPython Experiment1 >:	stop: phase=experiment
{'message': 'Completed experiment: iPython Experiment1  with execution time: 0.7934765815734863'}
{'message': 'Executing experiment: IPython Experiment 3'}
{'Step_0': {'algorithm': {'attributes': {'path': '.steps[0][1]'},
                          'value': 'principal component analysis'},
            'doc': {'attributes': {'path

INFO: 2019-02-04 13:17:04.948 		Split<id:0_c81dde85-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-02-04 13:17:04.949 		Split<id:0_c81dde85-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-02-04 13:17:04.949 	Run<id:c81dde84-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-02-04 13:17:04.950 	Run<id:c81dde84-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-02-04 13:17:04.951 	Run<id:c81dde86-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-02-04 13:17:04.952 	Run<id:c81dde86-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-02-04 13:17:04.953 		Split<id:0_c81dde87-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-02-04 13:17:04.953 		Split<id:0_c81dde87-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-02-04

INFO: 2019-02-04 13:17:05.160 		Split<id:0_c81dde9f-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-02-04 13:17:05.161 		Split<id:0_c81dde9f-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-02-04 13:17:05.161 	Run<id:c81dde9e-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-02-04 13:17:05.161 	Run<id:c81dde9e-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-02-04 13:17:05.163 	Run<id:c81ddea0-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-02-04 13:17:05.163 	Run<id:c81ddea0-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-02-04 13:17:05.164 		Split<id:0_c81ddea1-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-02-04 13:17:05.164 		Split<id:0_c81ddea1-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-02-04

INFO: 2019-02-04 13:17:05.361 	Run<id:c81ddeb8-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-02-04 13:17:05.362 	Run<id:c81ddeb8-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-02-04 13:17:05.363 		Split<id:0_c81ddeb9-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-02-04 13:17:05.364 		Split<id:0_c81ddeb9-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-02-04 13:17:05.378 		Split<id:0_c81ddeb9-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-02-04 13:17:05.379 		Split<id:0_c81ddeb9-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-02-04 13:17:05.379 	Run<id:c81ddeb8-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-02-04 13:17:05.379 	Run<id:c81ddeb8-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-02-04

INFO: 2019-02-04 13:17:05.572 		Split<id:0_c81dded1-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-02-04 13:17:05.573 		Split<id:0_c81dded1-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	stop: phase=split
INFO: 2019-02-04 13:17:05.573 	Run<id:c81dded0-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-02-04 13:17:05.573 	Run<id:c81dded0-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	stop: phase=run
INFO: 2019-02-04 13:17:05.575 	Run<id:c81dded2-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-02-04 13:17:05.576 	Run<id:c81dded2-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	start: phase=run
INFO: 2019-02-04 13:17:05.576 		Split<id:0_c81dded3-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-02-04 13:17:05.577 		Split<id:0_c81dded3-2876-11e9-bfa3-1860248c6d22;name:IPython Experiment 3>:	start: phase=split
INFO: 2019-02-04

In [27]:
# Go online, register new user
# For registering, user has to use the browser to enter details

In [20]:
# Log in to server
# Implemented

In [21]:
# Upload/Sync experiments on demand
# Not Implemented. Experiments can only be uploaded during the execution of the experiment

In [22]:
# Look at the experiment via SPA and check the results and visualize them(if possible)

In [23]:
# Download an experiment from the server and run it.(via the experiment.JSON file) Example for reproducibility
# Not yet implemented

In [ ]:
# Keep track of random seeds used in the experiment
# Not yet implemented

In [ ]:
# Download a dataset from OpenML and upload it to the server
# Generate metrics for uploaded  datasets
# Currently in progress. Some datasets in OpenML do not work and raise exceptions

In [ ]:
# Download a dataset from server
# Implemented

In [ ]:
# Running an experiment with a dataset downloaded from the server and sync the experiment automatically
# Not implemented

In [ ]:
# Running an experiment on the server cluster

In [ ]:
# Tracking the progress of experiments including hyperparameter search
# Code to be included in the backend for logging the porogress

In [ ]:
# Visualizing the results of an experiment/ dataset